In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1492 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What event marked the fall of the Western Roman Empire?","answers":[{"answer":"Sack of Rome","correct":false},{"answer":"Battle of Adrianople","correct":false},{"answer":"Fall of Constantinople","correct":false},{"answer":"476 AD","correct":true}]}]}'

In [2]:
import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '476 AD', 'correct': False}, {'answer': '1492 AD', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}]}
{'question': 'What event marked the fall of the Western Roman Empire?', 'answers': [{'answer': 'Sack of Rome', 'correct': False}, {'answer': 'Battle of Adrianople', 'correct': False}, {'answer': 'Fall of Constantinople', 'correct': False}, {'answer': '476 AD', 'correct': True}]}


In [ ]:
# function 정의 추가
function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

# llm 설정 업데이트
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo-1106",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
).bind(
    function_call={"name": "create_quiz"},
    functions=[function],
)

# formatting_chain 수정
formatting_chain = formatting_prompt | llm


# run_quiz_chain 함수 수정
@st.cache_data(show_spinner="Making quiz...")
def run_quiz_chain(_docs, topic):
    chain = {"context": questions_chain} | formatting_chain
    response = chain.invoke(_docs)
    return json.loads(response.additional_kwargs["function_call"]["arguments"])